In [ ]:
import os
import pandas as pd


BASE_DATASET_PATH = "../dataset/clothing-dataset"
DATASET_CSV_PATH = os.path.join(BASE_DATASET_PATH, "images.csv")
DATASET_IMAGES_PATH = os.path.join(BASE_DATASET_PATH, "images")

Нужно сэмплировать по 1 картинке каждого класса для тестовых данных и создать тестовую `.csv`-шку.

In [ ]:
df = pd.read_csv(DATASET_CSV_PATH)
df.head()

Давайте проверим, что все картинки из `.csv` файла действительно существуют в датасете.

In [ ]:
df["exists"] = df["image"].apply(
    lambda image_hash: os.path.exists(os.path.join(DATASET_IMAGES_PATH, image_hash + ".jpg"))
)
df["exists"].value_counts()

5 картинок, как видно, в действительности в датасете нет. Давайте от них избавимся.

In [ ]:
df = df[df["exists"]].reset_index(drop=True)
assert df["exists"].all()
df.drop("exists", axis=1, inplace=True)
df.head(3)

Давайте сразу же сохраним исправленный датафрейм вместо исходного, чтобы в процессе обучения не столкнуться с проблемой несуществующих картинок.

In [5]:
df.to_csv(DATASET_CSV_PATH, index=False)

Теперь сформируем маленький набор данных для тестов:
1. Возьмем по 1 картинке из каждого класса;
2. Сформируем маленький `.csv`-файлик для тестов.

In [6]:
test_csv = df.groupby("label").sample(1).reset_index(drop=True)
test_csv.to_csv("../test_data/test.csv", index=False)

In [7]:
import shutil

os.makedirs("../test_data/images/", exist_ok=True)
for item in test_csv.itertuples():
    src_path = os.path.join(DATASET_IMAGES_PATH, item.image + ".jpg")
    dst_path = os.path.join("../test_data/images", item.image + ".jpg")
    shutil.copy(src_path, dst_path)

Убедимся, что картинки скопировались.

In [8]:
import cv2


for item in test_csv.itertuples():
    image_path = os.path.join("../test_data/images", item.image + ".jpg")
    image = cv2.imread(image_path)
    assert image is not None